In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))
import tensorflow as tf
from policy import ConvNet
from concepts import DynamicConcepts, linear_regression
import absl.logging
import json
import string
import random
import numpy as np
import os
from env import gogame
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

absl.logging.set_verbosity(absl.logging.ERROR)

# Set memory growth
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

session_name = "falcon"
model_name = "net"
board_size = 5
board_name = f'{board_size}x{board_size}'
agents_to_sample = [10, 20, 60, 100, 500, 1000]

# Mcts simulations
simulations = 1000

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/board_size_{board_size}/net_1000.keras"

init_state, concept_type_single = DynamicConcepts.opening_play(board_size)

dynamic_concept = DynamicConcepts(
    init_state, simulations, board_size, concept_type_single, mcts_model_path)

CONCEPT_NAME = DynamicConcepts.opening_play.__name__

2024-02-09 14:07:44.365166: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-09 14:07:46.998256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-09 14:07:46.998297: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp5q5zkjw0/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp5q5zkjw0/assets
2024-02-09 14:07:51.395092: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp5q5zkjw0
2024-02-09 14:07:51.397373: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-09 14:07:51.397388: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp5q5zkjw0
2024-02-09 14:07:51.406316: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-09 14:07:51.451880: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp5q5zkjw0
2024-02-09 14:07:51.467658: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 72569 microseconds.
INFO: Crea

In [9]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Optimal child: 9
Optimal child: 0
Optimal child: 0
Optimal child: 24
Optimal child: 0
Optimal child: 0
Optimal child: 1
Optimal child: 0


In [10]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 5, 5)
Negative cases:  (8, 5, 5, 5)


In [5]:
# If larger than 50, do not visualize the tree do to the size (crashes the kernel)
if simulations <= 100:
    graph = dynamic_concept.view_tree()

    graph.render('./visualization/images/tree', view=True)

In [6]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [7]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print(gogame.str(positive_cases[i], nn_format=True))
    print()

In [ ]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print(gogame.str(negative_cases[i], nn_format=True))
    print()

In [12]:
BINARY = True

# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=50,
    dynamic=True
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Positions to consider:  19
Epoch 1/50


2024-02-09 14:10:05.522355: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:10:05.561937: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 1s/step - loss: 0.8694
Epoch 2/50
1/1 [==============================] - 0s 24ms/step - loss: 0.8689
Epoch 3/50
1/1 [==============================] - 0s 24ms/step - loss: 0.8684
Epoch 4/50
1/1 [==============================] - 0s 23ms/step - loss: 0.8679
Epoch 5/50
1/1 [==============================] - 0s 23ms/step - loss: 0.8673
Epoch 6/50
1/1 [==============================] - 0s 23ms/step - loss: 0.8668
Epoch 7/50
1/1 [==============================] - 0s 25ms/step - loss: 0.8663
Epoch 8/50
1/1 [==============================] - 0s 22ms/step - loss: 0.8658
Epoch 9/50
1/1 [==============================] - 0s 23ms/step - loss: 0.8653
Epoch 10/50
1/1 [==============================] - 0s 22ms/step - loss: 0.8648
Epoch 11/50
1/1 [==============================] - 0s 23ms/step - loss: 0.8643
Epoch 12/50
1/1 [==============================] - 0s 22ms/step - loss: 0.8638
Epoch 13/50
1/1 [==============================] - 0s 24ms/step - loss: 0.

2024-02-09 14:10:07.780457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [14]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=100,
            dynamic=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 1/1 [00:00<00:00, 91.59it/s]

Performing regression for layer 0
Epoch 1/100



2024-02-09 14:11:47.646881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:11:47.686656: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 676ms/step - loss: 1.1214
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 1.1200
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1186
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1172
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1159
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1145
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1131
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1118
Epoch 9/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1104
Epoch 10/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1090
Epoch 11/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1077
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1064
Epoch 13/100
1/1 [==============================] - 0s 22ms/

2024-02-09 14:11:50.595202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:11:50.942706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:11:50.975189: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-09 14:12:43.288838: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:12:43.642468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:12:43.674900: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 565ms/step - loss: 1.1558
Epoch 2/100
1/1 [==============================] - 0s 58ms/step - loss: 1.1528
Epoch 3/100
1/1 [==============================] - 0s 39ms/step - loss: 1.1498
Epoch 4/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1468
Epoch 5/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1438
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1409
Epoch 7/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1379
Epoch 8/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1350
Epoch 9/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1321
Epoch 10/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1292
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1263
Epoch 12/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1235
Epoch 13/100
1/1 [==============================] - 0s 23ms/

2024-02-09 14:12:46.463124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:12:46.791876: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-09 14:12:46.824502: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


1/1 [==============================] - 1s 571ms/step - loss: 1.1486
Epoch 2/100
1/1 [==============================] - 0s 59ms/step - loss: 1.1461
Epoch 3/100
1/1 [==============================] - 0s 40ms/step - loss: 1.1437
Epoch 4/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1412
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1388
Epoch 6/100
1/1 [==============================] - 0s 23ms/step - loss: 1.1363
Epoch 7/100
1/1 [==============================] - 0s 22ms/step - loss: 1.1339
Epoch 8/100
1/1 [==============================] - 0s 25ms/step - loss: 1.1315
Epoch 9/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1291
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1267
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1243
Epoch 12/100
1/1 [==============================] - 0s 24ms/step - loss: 1.1219
Epoch 13/100
1/1 [==============================] - 0s 23ms/

2024-02-09 14:12:49.669999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
